# Import package

In [24]:
!pip install wandb
!pip install xtarfile
!pip install pyenchant

In [25]:
START_TOKEN="\t"
END_TOKEN="\n"

In [26]:
import os
import random
import time
import wandb
import re, string
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from wordcloud import WordCloud, STOPWORDS
from collections import Counter


import os
from os.path import exists
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
import numpy as np
import keras
import tensorflow as tf
import csv

from tensorflow.keras import layers
import tensorflow.keras.backend as K




In [27]:

from Utility.Encoder  import Encoder
from Utility.Decoder  import Decoder
from Utility.Attention  import BahdanauAttention
from Utility.SeqTOSeq  import SequenceTOSequence
from Utility.Param  import Parameters
from Utility.DataLoader import downloadDataSet,get_files,tokenize,preprocess_data

In [28]:
import wandb 

In [29]:
wandb.init()

# Data Preprocessing

In [30]:

START_TOKEN="\t"
END_TOKEN="\n"


In [31]:
downloadDataSet()

You are all set


In [32]:
language="te"
train_dir, val_dir, test_dir = get_files(language)

dataset, input_tokenizer, targ_tokenizer = preprocess_data(train_dir)
val_dataset, _, _ = preprocess_data(val_dir)


In [33]:
#train data 
dataset, input_tokenizer, targ_tokenizer = preprocess_data(train_dir)

## Sweep Configuration

In [34]:
sweep_config = {
  "name": "DL_Assignment3_Rnn",
  "method": "bayes",
  "metric": {
      "name": "val_acc",
      "goal": "maximize",
  },
  
  "parameters": {
        "num_of_encoders":{
          "values":[1,2,3]    
        },
        "num_of_decoders":{
            "values":[1,2,3]      

        },
        "cell_type":{
          "values":['lstm','gru','rnn']  
        },
        
        "lr":{
          "values":[0.001,0.005,0.05]  
        },
        "optimizer":{
          "values":['adam','nadam','rmsprop']  
        },
        "dropout":{ "values": [0.25,0.3,0.4,0.5]},
        "latent_dim":{ "values": [128,256,512]},
        "inp_emb_size": {"values": [128,256]},
        "batch_size":{"values":[128,256]},
        
        }
    }

In [35]:


# This is the main function to use to train/fine-tune the model using wandb runs
def train_wandb():
    run = wandb.init()
  
    config=wandb.config
    # Set the run name
    name="num_of_encoders("+ str(config["num_of_encoders"]) + ")_"
    name = " num_of_decoders(" + str(config["num_of_decoders"]) + ")_"
    name += " cell_type(" + str(config["cell_type"]) + ")_"
    name += "latent_dim(" + str(config["latent_dim"])+ ")_"
    name += "lr(" + str(config["lr"])+ ")_"
    name += "optimizer(" + str(config["optimizer"]) + ")_"
    name += "dropout(" + str(config["dropout"]) + ")"
    name += "inp_emb_size(" + str(config["inp_emb_size"]) + ")_"
    name+="batch_size(" + str(config["batch_size"]) + ")"


    wandb.run.name = name[:-1]
    batch_size=config["batch_size"]
    inp_emb_size=config["inp_emb_size"]
    dropout=config["dropout"]
    optimizer=config["optimizer"]

    num_of_encoders=config["num_of_encoders"]
    num_of_decoders=config["num_of_decoders"]

    lr=config["lr"]
    latent_dim=config["latent_dim"]
    cell_type=config["cell_type"]

   
    param=Parameters(language="te",\
                        embedding_dim=inp_emb_size,\
                        encoder_layers=num_of_encoders,\
                        decoder_layers=num_of_decoders,\
                        layer_type=cell_type,\
                        units=latent_dim,\
                        dropout=dropout,
                        epochs=25,\
                 batch_size=batch_size\
                   )
    param.apply_teacher_forcing=True
    param.teacher_forcing_ratio=1
    param.patience=7
    model = SequenceTOSequence(param)
    model.set_vocabulary(input_tokenizer, targ_tokenizer)
    model.build(loss=tf.keras.losses.SparseCategoricalCrossentropy(),\
                metric = tf.keras.metrics.SparseCategoricalAccuracy(),\
                optimizer = optimizer,\
                lr=lr\
                )

    model.fit(dataset, val_dataset, epochs=param.epochs, wandb=wandb,teacher_forcing_ratio=param.teacher_forcing_ratio)       
    



In [36]:
           

sweep_id = wandb.sweep(sweep_config, entity="kankan-jana", project="CS6910_Assignment3")
    

Create sweep with ID: lwqft9sb
Sweep URL: https://wandb.ai/kankan-jana/CS6910_Assignment3/sweeps/lwqft9sb


In [37]:

wandb.agent(sweep_id, train_wandb)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 77wun3hj with config:
wandb: 	batch_size: 256
wandb: 	cell_type: lstm
wandb: 	dropout: 0.5
wandb: 	inp_emb_size: 256
wandb: 	latent_dim: 256
wandb: 	lr: 0.001
wandb: 	num_of_decoders: 2
wandb: 	num_of_encoders: 1
wandb: 	optimizer: rmsprop


wandb: ERROR Run 77wun3hj errored: AttributeError("'Parameters' object has no attribute 'param'")
wandb: Ctrl + C detected. Stopping sweep.
